# Voltage dependant measurement analysis
* This notebook is for analysis of v oltage dependant data taken from the PLQe imaging setup. 
* Assumes pre processing has already been done
    
### Some instructions: 
* Please run each cell in order, following the instructions
    * Press cntrl+enter to run a cell
* When prompted enter information into the feilds and press enter
* If you go back and re run a cell, that's fine but you have to run every cell after that again, and in order

This might break if all these files are not there! If you want to edit this code definately go ahead, a bunch of the functions live in seperate .py files, BUT please make a copy!!!

In [ ]:
from external_imports import *
from image_process import *
from general_data_process import *
from voltage_dependant import *

# For parallel processing
from joblib import Parallel, delayed
import multiprocessing
num_cores = multiprocessing.cpu_count()


import matplotlib
matplotlib.rcParams['font.size'] = 12
matplotlib.rcParams['font.family'] = "Century Gothic"

## Core process
* To do anything we need the PLQE maps, so let's make those first
* Follow these steps:

#### STEP 1: Determine what data we have
* Provide a path to the data folder and folder you would like to save everything in
    * Just copy and paste paths as raw strings (r"path\to\data") into approriate fields 
* Then, for each substrate, provide the bandgap in the input fields
* If you have alreay saved the PLQE maps before and are just re-running this book just run this cell to remind the kernal where all the files live

In [ ]:
datapath = r""
savepath = r""
if not os.path.isdir(savepath):
    os.makedirs(savepath)

path_db = path_process(datapath)

bandgaps = {}
for key in path_db.keys():
    if key.lower() != 'white':
        string = "What's the bandgap of "+key+" in eV?: "
        bandgap = float(input(string))
        bandgaps[key] = bandgap

#### STEP 2: Calculate PLQEs
* Now just run the following to save the PLQE maps. 

In [ ]:
whiteparamsfile = f"{datapath}/white_params.csv"  # defines location of white ref parameter file
vsweep_folders = ['vsweep', 'vsweep_f', 'vsweep_b']  # For getting a flux at one sun

for key in path_db.keys():
    bandgap = bandgaps[key]
    for pix in path_db[key]: 
        whitefilename = f"{datapath}/{key}/{pix}/white.npy"
        
        # extracting flux at one sun (defined as flux of vsweep)
        flux1sun = 0
        for vsweep_folder in vsweep_folders:
            if os.path.isdir(f"{datapath}/{key}/{pix}/{vsweep_folder}"):
                flux1sun = float(find_npy(f"{datapath}/{key}/{pix}/{vsweep_folder}")[0].split('_')[2])
                break
        
        # save PLQEs for oc and sc
        for vsweep_folder in vsweep_folders:
            datapath_i = f"{datapath}/{key}/{pix}/{vsweep_folder}"
            if os.path.isdir(datapath_i):
                savepath_i = f"{savepath}/{key}/{pix}"
                if os.path.isdir(datapath_i):
                    save_PLQE(datapath_i, savepath_i, whitefilename, whiteparamsfile, bandgap, 
                              flux1sun=flux1sun, savename=vsweep_folder)

* Plots (png files for easy viewing):

In [ ]:
# Run this to save the images
vsweep_folders = ['vsweep', 'vsweep_f', 'vsweep_b']  # For getting a flux at one sun

for key in path_db.keys():
    for pix in path_db[key]:
        for vsweep_folder in vsweep_folders:
            path_i = f"{savepath}/{key}/{pix}/PLQE_{vsweep_folder}"
            if os.path.isdir(path_i):
                if not os.path.isdir(f"{path_i}/pngs"):
                    os.makedirs(f"{path_i}/pngs")

                filenames = find_npy(path_i)

                def process_one_file(filename):
                    savefile_name = '_'.join(filename.split('_')[:-1])+'.png'
                    bias = float(filename.split('_')[0])
                    
                    fig, ax = plt.subplots()
                    imarr = np.load(f"{path_i}/{filename}")
                    im = ax.imshow(imarr*100, cmap="inferno", vmin=np.mean(imarr*100)-3*np.std(imarr*100),
                                                              vmax=np.mean(imarr*100)+3*np.std(imarr*100))
                    
                    ax.set_title(f'V = {bias}')
                    cbar = plt.colorbar(im,pad=0.01,aspect=20)
                    cbar.set_label('PLQE (%)', rotation=270,labelpad=25)

                    scalebar_size = 0.1 # cm
                    scalebar_offset = 10 # pix

                    scalebar_width_pix = 0.1*pixels_per_cm
                    scalebar_height_pix = scalebar_width_pix/10
                    scalbar_y_position = imarr.shape[0]-scalebar_offset-scalebar_height_pix

                    text_x = scalebar_offset + scalebar_width_pix/2
                    text_y = scalbar_y_position

                    scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
                    ax.add_patch(scalebar)
                    txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
                    txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

                    plt.axis('off')

                    plt.savefig(f"{path_i}/pngs/{savefile_name}", dpi=300)
                    plt.close('all')
                Parallel(n_jobs=num_cores)(delayed(process_one_file)(filename) for filename in filenames)

## Different analysis modules:
* The following will be different cells that do different bits of analysis, just run the ones you want

### QFLS at different bias
* Plots maps of QFLS at different bias
* Saves in new folder in pix folder
* Draws from PLQE_vsweep[f/b] folder, so that needs to be present

In [ ]:
for key in path_db.keys():
    for pix in path_db[key]:
        path_i = f"{savepath}/{key}/{pix}"
        voc_rad = vocradf(bandgaps[key])
        vsweep_array_QFLS(path_i, voc_rad)

* Plots:

In [ ]:
vsweep_folders = ['vsweep', 'vsweep_f', 'vsweep_b']  # For getting a flux at one sun

for key in path_db.keys():
    for pix in path_db[key]:
        for vsweep_folder in vsweep_folders:
            path_i = f"{savepath}/{key}/{pix}/QFLS_{vsweep_folder}"
            if os.path.isdir(path_i):
                if not os.path.isdir(f"{path_i}/pngs"):
                    os.makedirs(f"{path_i}/pngs")

                filenames = find_npy(path_i)

                def process_one_file(filename):
                    savefile_name = '_'.join(filename.split('_')[:-1])+'.png'
                    bias = float(filename.split('_')[0])
                    
                    fig, ax = plt.subplots()
                    imarr = np.load(f"{path_i}/{filename}")
                    im = ax.imshow(imarr, cmap="inferno", vmin=np.mean(imarr)-3*np.std(imarr),
                                                              vmax=np.mean(imarr)+3*np.std(imarr))
                    
                    ax.set_title(f'V = {bias}')
                    cbar = plt.colorbar(im,pad=0.01,aspect=20)
                    cbar.set_label('QFLS (eV)', rotation=270,labelpad=25)

                    scalebar_size = 0.1 # cm
                    scalebar_offset = 10 # pix

                    scalebar_width_pix = 0.1*pixels_per_cm
                    scalebar_height_pix = scalebar_width_pix/10
                    scalbar_y_position = imarr.shape[0]-scalebar_offset-scalebar_height_pix

                    text_x = scalebar_offset + scalebar_width_pix/2
                    text_y = scalbar_y_position

                    scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
                    ax.add_patch(scalebar)
                    txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
                    txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

                    plt.axis('off')

                    plt.savefig(f"{path_i}/pngs/{savefile_name}", dpi=300)
                    plt.close('all')
                Parallel(n_jobs=num_cores)(delayed(process_one_file)(filename) for filename in filenames)

### Collection efficienct
* Plots colection efficiency :
($\frac{PL_{oc} - PL_{sc}}{PL_{oc}}$)
* Draws from PLQE_vsweep[f/b] folder, so that needs to be present

In [ ]:
for key in path_db.keys():
    for pix in path_db[key]:
        path_i = f"{savepath}/{key}/{pix}"
        rawpath_i = f"{datapath}/{key}/{pix}"
        vsweep_col_eff(path_i, rawpath_i)                

* Plots:

In [ ]:
vsweep_folders = ['vsweep', 'vsweep_f', 'vsweep_b']  # For getting a flux at one sun

for key in path_db.keys():
    for pix in path_db[key]:
        for vsweep_folder in vsweep_folders:
            path_i = f"{savepath}/{key}/{pix}/coleff_{vsweep_folder}"
            if os.path.isdir(path_i):
                if not os.path.isdir(f"{path_i}/pngs"):
                    os.makedirs(f"{path_i}/pngs")

                filenames = find_npy(path_i)

                def process_one_file(filename):
                    savefile_name = '_'.join(filename.split('_')[:-1])+'.png'
                    bias = float(filename.split('_')[0])
                    
                    fig, ax = plt.subplots()
                    imarr = np.load(f"{path_i}/{filename}")
                    im = ax.imshow(imarr*100, cmap="inferno", vmin=np.mean(imarr*100)-3*np.std(imarr*100),
                                                              vmax=np.mean(imarr*100)+3*np.std(imarr*100))
                    
                    ax.set_title(f'V = {bias}')
                    cbar = plt.colorbar(im,pad=0.01,aspect=20)
                    cbar.set_label('collection efficiency (eV)', rotation=270,labelpad=25)

                    scalebar_size = 0.1 # cm
                    scalebar_offset = 10 # pix

                    scalebar_width_pix = 0.1*pixels_per_cm
                    scalebar_height_pix = scalebar_width_pix/10
                    scalbar_y_position = imarr.shape[0]-scalebar_offset-scalebar_height_pix

                    text_x = scalebar_offset + scalebar_width_pix/2
                    text_y = scalbar_y_position

                    scalebar = patches.Rectangle((scalebar_offset,scalbar_y_position),scalebar_width_pix,scalebar_height_pix,linewidth=1,edgecolor='k',facecolor='white')
                    ax.add_patch(scalebar)
                    txt = plt.text(text_x,text_y,"1 mm", color = "white", ha='center', va='bottom', fontweight='bold')
                    txt.set_path_effects([PathEffects.withStroke(linewidth=1.2, foreground='k')])

                    plt.axis('off')

                    plt.savefig(f"{path_i}/pngs/{savefile_name}", dpi=300)
                    plt.close('all')
                Parallel(n_jobs=num_cores)(delayed(process_one_file)(filename) for filename in filenames)